In [1]:
import csv
def intRow(row):
    temp = []
    for t in row:
        temp.append(int(t))
    return temp

def read_x(address):
    reader = csv.reader(open(address))
    x= [intRow(row) for row in reader]
    return x

def read_y(address):
    reader = csv.reader(open(address))
    list=[intRow(row) for row in reader]
    y=sum(list,[])
    return y

def NormalLine(row):
    total = sum(row[-4:])
    if total == 0:
        total = 1
    return [i/total for i in row]

def differences(row):
    total_list = row[-4:]
    num_list = row[:-4]
    differences_list = []
    for i in range(int(len(num_list)/4)-1):
        t1=num_list[i*4:(i+1)*4]
        t2=num_list[(i+1)*4:(i+2)*4]
        differences_list += [t2[j]-t1[j] for j in range(len(t1))]
       
    return num_list + differences_list +total_list


In [2]:
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE 
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier

def get_metrics(y_true,y_pre):    
    result1=[recall_score(y_true, y_pre),
             precision_score(y_true, y_pre),
             f1_score(y_true, y_pre)]
    
    y_flip_true = [0 if i else 1 for i in y_true]
    y_flip_pre = [0 if i else 1 for i in y_pre]
    
    result2=[recall_score(y_flip_true, y_flip_pre),
             precision_score(y_flip_true, y_flip_pre),
             f1_score(y_flip_true, y_flip_pre)]
    
    return result1,result2

def knn_best_parameter(X_original,Y,random_seed):
    x_normal = [NormalLine(row) for row in X_original]
    x_diff = [differences(row) for row in X_original]
    x_diff_normal = [differences(NormalLine(row)) for row in X_original]
    
    X_list = [X_original,x_normal,x_diff,x_diff_normal]
    
    total_result =[]
    for X in X_list:
        x_random_train, x_test, y_random_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=random_seed)
        rus = RandomUnderSampler(random_state=random_seed)
        x_under_train, y_under_train = rus.fit_resample(x_random_train, y_random_train)
        sm = SMOTE(random_state=random_seed)
        x_smote_train, y_smote_train = sm.fit_resample(x_random_train, y_random_train)
        
        train_list = [(x_random_train,y_random_train),(x_under_train,y_under_train),(x_smote_train, y_smote_train)]
        
        print("train")
        for x_train,y_train in train_list:
            dead_result_total =[]
            alive_result_total =[]
            for criterion in ['gini', 'entropy']:
                for n in [10,50,100,200]:
                    rf = RandomForestClassifier(n_estimators=n,criterion=criterion)
                    rf.fit(x_train,y_train)
                    y_pre = rf.predict(x_test)
                    dead_result,alive_result = get_metrics(y_test,y_pre)
                    dead_result_total +=dead_result
                    alive_result_total += alive_result
            total_result.append(dead_result_total)
            total_result.append(alive_result_total)
    
    return total_result

In [6]:
random_seed = 54321
X_address = 'X_m.csv'
Y_address = 'Y_m.csv'

root = "./result/rf/"
result_address = 'rf_m_'+str(random_seed)+'.csv'

In [7]:
X = read_x(X_address)
Y = read_y(Y_address)

result = knn_best_parameter(X,Y,random_seed)

print(len(result))
print(len(result[0]))

train
train
train
train
24
24


In [8]:
with open(root+result_address, 'w',newline='') as f:
    print("writing..")
    writer = csv.writer(f)
    for row in result:
        writer.writerow(row)
print("finish..")



writing..
finish..
